In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from os import listdir, path
from sklearn.model_selection import train_test_split

from data_utils import PCA, RandomForestClassifier, StandardScaler, SVC
from data_utils import classification_error, display_confusion_matrix
from data_utils import LFWUtils

from image_utils import make_image, open_image

from PIL import Image

In [ ]:
# 设置图像路径 # Set the image path
data_dir = "data/image"
categories = ['family', 'hello', 'help', 'house', 'i_love_you', 'no', 'please', 'sorry', 'thankyou', 'yes']

In [ ]:
# 加载数据 #load data
def load_images(data_dir, categories):
    data = []
    labels = []
    for label, category in enumerate(categories):
        category_path = os.path.join(data_dir, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            try:
                # Open the image and grayscale it
                img = Image.open(img_path).convert('L')
                img = img.resize((64, 64))  # 调整到固定大小
                data.append(np.array(img).flatten())  # 展平图像
                labels.append(label)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(data), np.array(labels)

In [25]:
X, y = load_images(data_dir, categories)

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 特征降维
pca = PCA(n_components=50)  # 设置降维后的主成分数
X_pca = pca.fit_transform(X_scaled)

Exception: Input has wrong type. Please use pandas DataFrame